# 10 Taking Advantage of First Class Objects 

### 10.1 First Class Objects


Python exposes many language features and places
almost no constraints on what types data
structures can hold.

Here's an example of using a dictionary of functions to create a
simple calculator.  In some languages the only reasonable solution
would require a `case` or `switch` statement, or a series of `if`
statements.  If you've been using such a language for a while, this
example may help you expand the range of solutions you can imagine in
Python.

Let's iteratively write code to get this behaviour:

    assert calc('7+3') == 10
    assert calc('9-5') == 4
    assert calc('9/3') == 3


In [ ]:
7+3

In [ ]:
expr = '7+3'

In [ ]:
lhs, op, rhs = expr

In [ ]:
lhs, op, rhs

In [ ]:
lhs, rhs = int(lhs), int(rhs)

In [ ]:
lhs, op, rhs

In [ ]:
op, lhs, rhs

In [ ]:
def perform_operation(op, lhs, rhs):
    if op == '+':
        return lhs + rhs
    if op == '-':
        return lhs - rhs
    if op == '/':
        return lhs / rhs

In [ ]:
perform_operation('+', 7, 3) == 10

  The `perform_operation` function has a lot of boilerplate repetition.
Let's use a data structure instead to use less code and make it easier to extend.

In [ ]:
import operator

In [ ]:
operator.add(7, 3)

In [ ]:
OPERATOR_MAPPING = {
    '+': operator.add,
    '-': operator.sub,
    '/': operator.truediv,
    }

In [ ]:
OPERATOR_MAPPING['+']

In [ ]:
OPERATOR_MAPPING['+'](7, 3)

In [ ]:
def perform_operation(op, lhs, rhs):
    return OPERATOR_MAPPING[op](lhs, rhs)

In [ ]:
perform_operation('+', 7, 3) == 10

In [ ]:
def calc(expr):
    lhs, op, rhs = expr
    lhs, rhs = int(lhs), int(rhs)
    return perform_operation(op, lhs, rhs)

In [ ]:
calc('7+3')

In [ ]:
calc('9-5')

In [ ]:
calc('9/3')

In [ ]:
calc('3*4')

In [ ]:
OPERATOR_MAPPING['*'] = operator.mul

In [ ]:
calc('3*4')

Let's look at another example.  Suppose we have data where every
line is fixed length with fixed length records in it and we want to
pull fields out of it by name:

    PYTHON_RELEASES = [
        'Python 3.4.0 2014-03-17',
        'Python 3.3.0 2012-09-29',
        'Python 3.2.0 2011-02-20',
        'Python 3.1.0 2009-06-26',
        'Python 3.0.0 2008-12-03',
        'Python 2.7.9 2014-12-10',
        'Python 2.7.8 2014-07-02',
    ]

    release34 = PYTHON_RELEASES[0]

    release = ReleaseFields(release34)  # 3.4.0
    assert release.name == 'Python'
    assert release.version == '3.4.0'
    assert release.date == '2014-03-17'

  This works:

In [ ]:
class ReleaseFields:
    def __init__(self, data):
        self.data = data
    
    @property
    def name(self):
        return self.data[0:6]
    
    @property
    def version(self):
        return self.data[7:12]
    
    @property
    def date(self):
        return self.data[13:23]

In [ ]:
release34 = 'Python 3.4.0 2014-03-17'

In [ ]:
release = ReleaseFields(release34)

In [ ]:
assert release.name == 'Python'
assert release.version == '3.4.0'
assert release.date == '2014-03-17'

  However, the following is better especially if there are many fields
or as part of a libary which handle lots of different record formats:

In [ ]:
class ReleaseFields:
    slices = {
        'name': slice(0, 6),
        'version': slice(7, 12),
        'date': slice(13, 23)
        }
    
    def __init__(self, data):
        self.data = data
    
    def __getattr__(self, attribute):
        if attribute in self.slices:
            return self.data[self.slices[attribute]]
        raise AttributeError(
            "{!r} has no attribute {!r}"
            .format(self, attribute))

In [ ]:
release = ReleaseFields(release34)

In [ ]:
assert release.name == 'Python'
assert release.version == '3.4.0'
assert release.date == '2014-03-17'

  Confirm that trying to access an attribute that doesn't exist fails
correctly.  (Note they won't in Python 2.x unless you add `(object)`
after `class ReleaseFields`).

In [ ]:
release.foo == 'exception'

  If you find yourself writing lots of boilerplate code as in the
first versions of the calculator and fixed length record class
above, you may want to try changing it to use a Python data
structure with first class objects.

### 10.2 Binding Data with Functions

It is often useful to bind data to a function.  A method clearly
does that, binding the instance's attributes with the method behaviour,
but it's not the only way.

In [ ]:
def log(severity, message):
    print('{}: {}'.format(severity.upper(), message))

In [ ]:
log('warning', 'this is a warning')

In [ ]:
log('error', 'this is an error')

  Create a new function that specifies one argument.

In [ ]:
def warning(message):
    log('warning', message)

In [ ]:
warning('this is a warning')

  Create a closure from a function that specifies an argument.

In [ ]:
def create_logger(severity):
    def logger(message):
        log(severity, message)
    return logger

In [ ]:
warning2 = create_logger('warning')

In [ ]:
warning2('this is a warning')

  Create a partial function.

In [ ]:
import functools

In [ ]:
warning3 = functools.partial(log, 'warning')

In [ ]:
warning3

In [ ]:
warning3.func is log

In [ ]:
warning3.args, warning3.keywords

In [ ]:
warning3('this is a warning')

  Use a bound method.

In [ ]:
SENTENCE_PUNCUATION = '.?!'

In [ ]:
sentence = 'This is a sentence!'

In [ ]:
sentence[-1] in SENTENCE_PUNCUATION

In [ ]:
'.' in SENTENCE_PUNCUATION

In [ ]:
SENTENCE_PUNCUATION.__contains__('.')

In [ ]:
SENTENCE_PUNCUATION.__contains__(',')

In [ ]:
is_end_of_a_sentence = SENTENCE_PUNCUATION.__contains__

In [ ]:
is_end_of_a_sentence('.')

In [ ]:
is_end_of_a_sentence(',')

  Create a class with a `__call__` method.

In [ ]:
class SentenceEndsWith:
    def __init__(self, characters):
        self.punctuation = characters
    
    def __call__(self, sentence):
        return sentence[-1] in self.punctuation

In [ ]:
is_end_of_a_sentence_dot1 = SentenceEndsWith('.')

In [ ]:
is_end_of_a_sentence_dot1('This is a test.')

In [ ]:
is_end_of_a_sentence_dot1('This is a test!')

In [ ]:
is_end_of_a_sentence_any = SentenceEndsWith('.!?')

In [ ]:
is_end_of_a_sentence_any('This is a test.')

In [ ]:
is_end_of_a_sentence_any('This is a test!')

  Another way that mutable data can be bound to a function is with
parameter evaluation, which is sometimes done by mistake.

In [ ]:
def f1(parameter=print('The parameter is initialized now!')):
    if parameter is None:
        print('The parameter is None')
    return parameter

In [ ]:
f1()

In [ ]:
f1() is None

In [ ]:
f1('Not None')

In [ ]:
def f2(parameter=[0]):
    parameter[0] += 1
    return parameter[0]

In [ ]:
f2()

In [ ]:
f2()

In [ ]:
f2()

In [ ]:
f2()